In [31]:
import ase
import ase.io
from ase.io import xsf
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [32]:
with open ('/Users/mohammad/Documents/vdW-radii/Cloned-Repo/machine-learning-dimensionalities-and-binding-energy/data/export_structures_labeled.json') as f:
    labeled_structure = json.load(f)

In [33]:
len(labeled_structure)

5514

In [34]:
labeled_structure['22e4d077-bdc5-44ec-8aab-d94c9bd36b60']

{'cif_attributes': {'source': {'license': 'ICSD',
   'db_uri': None,
   'uri': 'http://theossrv2.epfl.ch/icsd/index.php?format=cif&action=Export&id%5B%5D=157091',
   'db_name': 'Icsd',
   'version': '2015.1',
   'id': '650091',
   'extras': {'idnum': '157091'}},
  'formulae': ['Re1 Se2'],
  'filename': 'tmp__WFGw',
  'md5': '2c1a986b627f6b26d177c89f7205f1bd'},
 'xsf': 'CRYSTAL\nPRIMVEC 1\n      3.3000000000       0.0000000000       0.0000000000\n     -1.6500000000       2.8578838325       0.0000000000\n      0.0000000000       0.0000000000      25.5000000000\nPRIMCOORD 1\n12 1\n75       0.0000000000       0.0000000000      12.7500000000\n75       1.6500000000       0.9526279442       0.0000000000\n75       1.6500000000       0.9526279442       6.3750000000\n75       1.6500000000       0.9526279442      19.1250000000\n34       0.0000000000       1.9052558883       8.1090000000\n34       0.0000000000       1.9052558883      17.3910000000\n34       0.0000000000       1.9052558883       4.

In [35]:
set_c = {}
for key in labeled_structure.keys():
    if len(labeled_structure[key]['attrs']['kinds']) == 1 and len(labeled_structure[key]['attrs']['sites']) < 100:
        set_c[key] = labeled_structure[key]
    elif len(labeled_structure[key]['attrs']['kinds']) in [2, 3] and len(labeled_structure[key]['attrs']['sites']) < 40:
        set_c[key] = labeled_structure[key]
    elif len(labeled_structure[key]['attrs']['kinds']) in [4, 5, 6] and len(labeled_structure[key]['attrs']['sites']) < 32:
        set_c[key] = labeled_structure[key]

In [36]:
len(set_c)

2923

In [37]:
structures_PassLowDim_NoDFT= []
for uuid in labeled_structure.keys():
    if uuid not in list(set_c.keys()):
        structures_PassLowDim_NoDFT.append(uuid)

In [38]:
len(structures_PassLowDim_NoDFT)

2591

In [39]:
uuids = []
structures = []

labels = []
number_of_atoms_in_bulks = []
number_of_highest_dimensionality_substructures_in_bulks = []
sub_structure_spgs = []
sub_structure_dims = []
energy_bulks = []
sub_structure_formulas = []
i = 0
for key in structures_PassLowDim_NoDFT:
    try:
        path = 'data/structures/structures_3D_xsf/'

        folder = os.fsencode(path)
        
        filename = key + '.xsf'
    
        with open(path + filename) as f:
            structure = ase.io.xsf.read_xsf(f, index=- 1, read_data=False)
        
        
        
        uuids.append(key)

        
        structures.append(structure)        

    except KeyError:
        continue
    except FileNotFoundError:
        i += 1

print('Number of missing structures:', i)

data_df2 = {
    'uuid' : uuids,
    'structure' : structures,
}

df = pd.DataFrame(data_df2)

Number of missing structures: 0


In [40]:
df

,uuid,structure
0,b4851513-c1f2-4f1b-b44b-586a730ccf81,"(Atom('K', [3.4373412417, 1.0473617, 7.0045109..."
1,2933936c-a017-4551-9f94-49bfbf4bc798,"(Atom('Li', [3.47775, 3.47775, 4.56945], index..."
2,25d2c223-6586-4dc7-89ac-735dce10f724,"(Atom('La', [2.721, 12.502, 0.0], index=0), At..."
3,1a6bcfa5-76c3-4a34-b234-222e603571c7,"(Atom('Ho', [1.6230062716, 3.5608609333, 2.748..."
4,d2338f69-8834-4555-b819-d0effbbd377e,"(Atom('In', [8.9788794007, 3.0105, 11.80533453..."
...,...,...
2586,a72e7a35-0f87-46a3-aab4-7d4c0fc1d450,"(Atom('Ta', [4.0121792363, 0.93425, 0.88138631..."
2587,78c43daa-5a10-41f5-a375-b74a3f0a99ca,"(Atom('Na', [-3.7997952728, 6.7265586, 11.9922..."
2588,c9e0705a-0db6-4ae0-952f-e72dde917094,"(Atom('Sm', [1.0174705925, 7.0451209, 3.305208..."
2589,eff7e985-82e9-4fa0-8160-14fbc9b18bdc,"(Atom('Tl', [0.0, 12.540798, 4.905], index=0),..."


In [41]:
from matminer.featurizers.conversions import ASEAtomstoStructure

In [42]:
aa2s = ASEAtomstoStructure()
df = aa2s.fit_featurize_dataframe(df, col_id= 'structure', ignore_errors=True)

ASEAtomstoStructure:   0%|          | 0/2591 [00:00<?, ?it/s]

In [43]:
df

,uuid,structure,PMG Structure from ASE Atoms
0,b4851513-c1f2-4f1b-b44b-586a730ccf81,"(Atom('K', [3.4373412417, 1.0473617, 7.0045109...","[[3.43734124 1.0473617 7.00451096] K, [4.4372..."
1,2933936c-a017-4551-9f94-49bfbf4bc798,"(Atom('Li', [3.47775, 3.47775, 4.56945], index...","[[3.47775 3.47775 4.56945] Li, [0. 0. ..."
2,25d2c223-6586-4dc7-89ac-735dce10f724,"(Atom('La', [2.721, 12.502, 0.0], index=0), At...","[[ 2.721 12.502 0. ] La, [0. 0. 0.] La, [ 0..."
3,1a6bcfa5-76c3-4a34-b234-222e603571c7,"(Atom('Ho', [1.6230062716, 3.5608609333, 2.748...","[[1.62300627 3.56086093 2.74825214] Ho, [8.393..."
4,d2338f69-8834-4555-b819-d0effbbd377e,"(Atom('In', [8.9788794007, 3.0105, 11.80533453...","[[ 8.9788794 3.0105 11.80533454] In, [7...."
...,...,...,...
2586,a72e7a35-0f87-46a3-aab4-7d4c0fc1d450,"(Atom('Ta', [4.0121792363, 0.93425, 0.88138631...","[[4.01217924 0.93425 0.88138632] Ta, [ 5.19..."
2587,78c43daa-5a10-41f5-a375-b74a3f0a99ca,"(Atom('Na', [-3.7997952728, 6.7265586, 11.9922...","[[-3.79979527 6.7265586 11.99229328] Na, [-1..."
2588,c9e0705a-0db6-4ae0-952f-e72dde917094,"(Atom('Sm', [1.0174705925, 7.0451209, 3.305208...","[[1.01747059 7.0451209 3.30520872] Sm, [ 5.37..."
2589,eff7e985-82e9-4fa0-8160-14fbc9b18bdc,"(Atom('Tl', [0.0, 12.540798, 4.905], index=0),...","[[ 0. 12.540798 4.905 ] Tl, [2.267 ..."


In [44]:
df.to_pickle("DataFrame_setC.pkl")

***Numbers of atoms***

In [45]:
df['num_atoms'] = df['PMG Structure from ASE Atoms'].apply(lambda x: x.composition.num_atoms)

In [47]:
df_Small_NumAtoms = df [ df['num_atoms'] < 41]

In [48]:
df_Small_NumAtoms.head()

,uuid,structure,PMG Structure from ASE Atoms,num_atoms
10,a48ba18b-47a0-4be2-8db7-5f1bb015d9ad,"(Atom('Hg', [4.3037781768, 1.184749484, 1.1790...","[[4.30377818 1.18474948 1.17906635] Hg, [-1.22...",32.0
23,a9409d85-176c-4b05-bc0e-5db9a558fdb9,"(Atom('V', [0.9956475435, 10.014224, 3.9075321...","[[ 0.99564754 10.014224 3.90753217] V, [2.6...",32.0
28,9960f669-1380-4f62-9f7e-61d60df36eb9,"(Atom('Tl', [3.5624, 2.056752599, 6.539959288]...","[[3.5624 2.0567526 6.53995929] Tl, [1.976...",36.0
32,ba5b0932-bc7c-4fbb-a090-63a754ae1072,"(Atom('Ce', [4.1819040786, 3.4505199099, 5.598...","[[4.18190408 3.45051991 5.5986254 ] Ce, [-0.83...",38.0
39,9e4dd954-deb9-4724-80d7-ebfdb64b2706,"(Atom('Dy', [-2.3936225648, 1.3152674807, 7.96...","[[-2.39362256 1.31526748 7.96038162] Dy, [4....",38.0


In [49]:
structures = list(df_Small_NumAtoms["PMG Structure from ASE Atoms"])
gridpoints = list(df_Small_NumAtoms.index)

In [50]:
features = {n:{} for n in gridpoints}
for name in gridpoints:
    features[name]["structure"]=df["PMG Structure from ASE Atoms"][name]

In [83]:
data=pd.DataFrame.from_dict(features).T
data

,structure
10,"[[4.30377818 1.18474948 1.17906635] Hg, [-1.22..."
23,"[[ 0.99564754 10.014224 3.90753217] V, [2.6..."
28,"[[3.5624 2.0567526 6.53995929] Tl, [1.976..."
32,"[[4.18190408 3.45051991 5.5986254 ] Ce, [-0.83..."
39,"[[-2.39362256 1.31526748 7.96038162] Dy, [4...."
...,...
2576,"[[2.21574279 4.9422198 3.97895321] Na, [ 1.32..."
2581,"[[5.09263524 3.50475 6.08172162] Ta, [ 1.92..."
2585,"[[4.4198 6.2288 2.064 ] Cu, [ 5.6252 12.0912 ..."
2586,"[[4.01217924 0.93425 0.88138632] Ta, [ 5.19..."


In [84]:
data = data.reset_index()
data

,index,structure
0,10,"[[4.30377818 1.18474948 1.17906635] Hg, [-1.22..."
1,23,"[[ 0.99564754 10.014224 3.90753217] V, [2.6..."
2,28,"[[3.5624 2.0567526 6.53995929] Tl, [1.976..."
3,32,"[[4.18190408 3.45051991 5.5986254 ] Ce, [-0.83..."
4,39,"[[-2.39362256 1.31526748 7.96038162] Dy, [4...."
...,...,...
355,2576,"[[2.21574279 4.9422198 3.97895321] Na, [ 1.32..."
356,2581,"[[5.09263524 3.50475 6.08172162] Ta, [ 1.92..."
357,2585,"[[4.4198 6.2288 2.064 ] Cu, [ 5.6252 12.0912 ..."
358,2586,"[[4.01217924 0.93425 0.88138632] Ta, [ 5.19..."


In [52]:
%matplotlib inline
from matplotlib import pyplot as plt
from matminer.datasets import load_dataset
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers.composition import ElementProperty, Stoichiometry, ValenceOrbital, IonProperty
from matminer.featurizers.structure import (SiteStatsFingerprint, StructuralHeterogeneity,
                                            ChemicalOrdering, StructureComposition, MaximumPackingEfficiency)
from matminer.featurizers.conversions import DictToObject
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ShuffleSplit, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from scipy import stats
from tqdm import tqdm_notebook as tqdm
import numpy as np

In [53]:
featurizer = MultipleFeaturizer([
    SiteStatsFingerprint.from_preset("CoordinationNumber_ward-prb-2017"),
    StructuralHeterogeneity(),
    ChemicalOrdering(),
    MaximumPackingEfficiency(),
    SiteStatsFingerprint.from_preset("LocalPropertyDifference_ward-prb-2017"),
    StructureComposition(Stoichiometry()),
    StructureComposition(ElementProperty.from_preset("magpie")),
    StructureComposition(ValenceOrbital(props=['frac'])),
    StructureComposition(IonProperty(fast=True))
])

In [54]:
dto = DictToObject(target_col_id='structure', overwrite_data=True)
data = dto.featurize_dataframe(data, 'structure')

DictToObject:   0%|          | 0/360 [00:00<?, ?it/s]

In [55]:
%%time
X = featurizer.featurize_many(data['structure'], ignore_errors=True)

MultipleFeaturizer:   0%|          | 0/360 [00:00<?, ?it/s]

CPU times: user 902 ms, sys: 386 ms, total: 1.29 s
Wall time: 14min 55s


In [85]:
df_features = pd.DataFrame(X)
df_features.head()

,0,1,2,3,4,5,6,7,8,9,...,263,264,265,266,267,268,269,270,271,272
0,6.802069,8.341539,1.539470,7.754725,0.367830,0.015961,1.039381,0.966159,0.134515,0.175084,...,130.000000,45.000000,70.0,0.126984,0.142857,0.396825,0.333333,True,0.080661,0.016833
1,4.549866,10.835070,6.285204,8.975368,1.589068,0.080245,1.079712,0.728332,0.068885,0.201405,...,83.250000,89.062500,12.0,0.241935,0.387097,0.370968,0.000000,True,0.559139,0.107182
2,5.747613,14.709639,8.962027,9.555633,1.953022,0.095777,1.320260,0.748328,0.072864,0.400766,...,58.500000,71.888889,12.0,0.246575,0.390411,0.267123,0.095890,True,0.563121,0.101023
3,6.726692,15.633412,8.906720,10.468351,1.972491,0.073900,1.239975,0.812006,0.045490,0.324571,...,66.105263,63.091413,12.0,0.250000,0.460526,0.276316,0.013158,True,0.739617,0.107661
4,6.225861,12.956786,6.730926,8.980190,1.836111,0.095098,1.141055,0.769876,0.008992,0.611368,...,68.105263,66.260388,12.0,0.255639,0.443609,0.225564,0.075188,True,0.708321,0.080819


In [90]:
df_features = pd.concat([data, df_features], axis = 1, ignore_index= False)
df_features.head()

,index,structure,0,1,2,3,4,5,6,7,...,263,264,265,266,267,268,269,270,271,272
0,10,"[[4.30377818 1.18474948 1.17906635] Hg, [-1.22...",6.802069,8.341539,1.539470,7.754725,0.367830,0.015961,1.039381,0.966159,...,130.000000,45.000000,70.0,0.126984,0.142857,0.396825,0.333333,True,0.080661,0.016833
1,23,"[[ 0.99564754 10.014224 3.90753217] V, [2.6...",4.549866,10.835070,6.285204,8.975368,1.589068,0.080245,1.079712,0.728332,...,83.250000,89.062500,12.0,0.241935,0.387097,0.370968,0.000000,True,0.559139,0.107182
2,28,"[[3.5624 2.0567526 6.53995929] Tl, [1.976...",5.747613,14.709639,8.962027,9.555633,1.953022,0.095777,1.320260,0.748328,...,58.500000,71.888889,12.0,0.246575,0.390411,0.267123,0.095890,True,0.563121,0.101023
3,32,"[[4.18190408 3.45051991 5.5986254 ] Ce, [-0.83...",6.726692,15.633412,8.906720,10.468351,1.972491,0.073900,1.239975,0.812006,...,66.105263,63.091413,12.0,0.250000,0.460526,0.276316,0.013158,True,0.739617,0.107661
4,39,"[[-2.39362256 1.31526748 7.96038162] Dy, [4....",6.225861,12.956786,6.730926,8.980190,1.836111,0.095098,1.141055,0.769876,...,68.105263,66.260388,12.0,0.255639,0.443609,0.225564,0.075188,True,0.708321,0.080819


In [91]:
df_features.to_pickle('features_MultipleFeaturizer_setC.pkl')

In [92]:
df_features = pd.read_pickle('features_MultipleFeaturizer_setC.pkl')